In [4]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import pandas as pd
import re

import numpy as np

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.metrics import roc_auc_score

import json
import warnings

import numpy as np
import pandas as pd

from category_encoders import OrdinalEncoder
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.svm import SVC
from scipy.stats import uniform
from sklearn.metrics import f1_score, accuracy_score

import matplotlib.pyplot as plt

In [5]:
df_train=pd.read_csv('./train_full1.csv')
df_train=df_train.drop(['Unnamed: 0','X','policy_id'],axis='columns')
df_train
df_train.isna().sum()

n_safty_rating      0
discount            0
Home_policy_ind     0
df_prior_carrier    0
df_cov_package      0
df_cat_zone         0
number_drivers      0
num_loaned_veh      0
num_owned_veh       0
num_leased_veh      0
driver_age          0
freq_agent          0
lon                 0
lat                 0
quoted_amt          0
credit_score        0
convert_ind         0
dtype: int64

In [6]:
df_train.isna().sum()

n_safty_rating      0
discount            0
Home_policy_ind     0
df_prior_carrier    0
df_cov_package      0
df_cat_zone         0
number_drivers      0
num_loaned_veh      0
num_owned_veh       0
num_leased_veh      0
driver_age          0
freq_agent          0
lon                 0
lat                 0
quoted_amt          0
credit_score        0
convert_ind         0
dtype: int64

In [7]:
df_test=pd.read_csv('./test_full1.csv')
df_test=df_test.drop(['Unnamed: 0','X','policy_id','convert_ind'],axis='columns')
df_test
df_test.isna().sum()


n_safty_rating      0
discount            0
Home_policy_ind     0
df_prior_carrier    0
df_cov_package      0
df_cat_zone         0
number_drivers      0
num_loaned_veh      0
num_owned_veh       0
num_leased_veh      0
driver_age          0
freq_agent          0
lon                 0
lat                 0
quoted_amt          0
credit_score        0
dtype: int64

In [6]:
len(df_test)

12291

In [8]:
df_train=df_train.drop(np.where(df_train.isna())[0],axis="rows")
df_train=df_train.reset_index()

df_test=df_test.drop(np.where(df_test.isna())[0],axis="rows")
df_test=df_test.reset_index()

In [9]:
x_train = df_train.drop(['convert_ind'], axis='columns')
y_train = df_train['convert_ind']
x = np.asarray(x_train)
y = np.asarray(y_train)


x_train_train, x_train_test, y_train_train, y_train_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [17]:
model = XGBClassifier(random_state=42, scale_pos_weight=8, eval_metric='auc')
model.fit(x_train_train, y_train_train)

result = model.predict_proba(x_train_test)

In [18]:
(model.predict(x_train_train))-y_train_train
roc_auc_score(model.predict(x_train_train),y_train_train)

0.6965617468254677

In [22]:
len(model.predict(df_test))

12291

## Cross validation

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

In [25]:
ind=np.random.randint(low=0,high=5,size=len(x_train))
auc=[]
for i in range(5):
    print(i)
    x_train_test=np.asarray(x_train.iloc[np.where(ind==i)[0]])
    x_train_train=np.asarray(x_train.iloc[np.where(ind!=i)[0]])
    y_train_test=np.asarray(y_train.iloc[np.where(ind==i)[0]])
    y_train_train=np.asarray(y_train.iloc[np.where(ind!=i)[0]])
    model = XGBClassifier(reg_alpha=2,reg_lambda=1,colsample_bytree=0.9,gamma=0.1,random_state=30, scale_pos_weight=4.5, eval_metric='auc',max_depth=3,learning_rate=0.1,n_estimators=200,min_child_weight=2.3,subsample=0.7)
    #model=LogisticRegression(penalty="l1",class_weight="balanced",solver="liblinear")
    #model=RandomForestClassifier(class_weight="balanced",max_depth=14,n_estimators=50,max_features=7,min_samples_leaf=1)
    #model=CatBoostClassifier(loss_function="CrossEntropy",eval_metric='AUC',depth=8)
    model.fit(x_train_train, y_train_train)
    auc.append(roc_auc_score(model.predict(x_train_test),y_train_test))
    
np.mean(auc)

0
1
2
3
4


0.5603665775108646

In [276]:
model = XGBClassifier(reg_alpha=2,reg_lambda=1,colsample_bytree=0.9,gamma=0.1,random_state=42, scale_pos_weight=4.5, eval_metric='auc',max_depth=3,learning_rate=0.1,n_estimators=240,min_child_weight=2.3,subsample=0.7)
model.fit(x,y)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.9,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='auc', feature_types=None, gamma=0.1, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=3, max_leaves=0, min_child_weight=2.3, missing=nan,
              monotone_constraints='()', n_estimators=240, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=42, ...)

In [23]:
sum(model.predict(np.asarray(df_test)))/len(df_test)

0.00040680172483931334

In [19]:
sum(y_train)/len(y)

0.11174093460985599